In [ ]:
import pathlib
import itertools
import collections
from typing import Dict, List, Union

import matplotlib.pylab as plt
from bokeh.plotting import figure, output_notebook, show
from ipywidgets import interact, interactive, fixed, IntSlider, Dropdown
import torch
import pandas as pd
from PIL import Image
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from dtreeviz.trees import *

from fastai.vision.all import *
from fastai.vision.widgets import *
from fastai.data.all import *
from fastai.tabular.all import *
from fastai.torch_core import set_seed

from scheeg import data
from scheeg.performance import accuracy, model_accuracy, sensitivity_specificity_auc

A snippet of code from [fastbook](https://github.com/fastai/fastbook) to draw a decision tree

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

def draw_tree(t, df, size=10, ratio=0.6, precision=0, **kwargs):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True, rounded=True,
                      special_characters=True, rotate=False, precision=precision, **kwargs)
    return graphviz.Source(re.sub('Tree {', f'Tree {{ size={size}; ratio={ratio}', s))

For the sake of reproducibility

In [ ]:
set_seed(42)

Some settings for *Pandas*

In [ ]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

Some colors to be used in LaTeX

In [ ]:
latex_colors = {
    'red',
    'Bittersweet',
    'blue',
    'Cerulean',
    'Violet',
    'Goldenrod',
    'ForestGreen',
#     'Gray',
    'YellowOrange',
    'RubineRed',
    'RoyalBlue',
    'Fuchsia',
    'OliveGreen',
    'Black'
}

# Random forests

> Experiments on...

## Parameters

The indexes of the subjects that are reserved for testing

In [ ]:
i_testing_subjects = [1, 2]

Those that, during training, will only be used for validation

In [ ]:
i_validation_subjects = [3, 4]

The dependent variable is named

In [ ]:
dep_var = 'ill'

The number of *folds* when performing cross-validation

In [ ]:
n_folds_cross_validation = 7

The path where results are to be saved

In [ ]:
output_path = pathlib.Path.home() / 'papers/frontiers_2020/results3'
output_path.mkdir(parents=True, exist_ok=True)
output_path

The names of the EEG channels

In [ ]:
eeg_channels_names = [
    'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'C3', 'Cz', 'C4',
    'P3', 'Pz', 'P4', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2']
assert len(eeg_channels_names) == 19

A mapping from indexes to names

In [ ]:
eeg_index_to_name = dict(list(enumerate(eeg_channels_names)))

## Data

The path to the MATLAB data file is assembled

In [ ]:
input_dir = pathlib.Path.cwd() / 'preprocessed_data'
input_file = input_dir / 'CON_MAT.mat'
assert input_file.exists()
input_file

*MATLAB* file is loaded

In [ ]:
metrics, (n_subjects, n_channels, _, n_frequency_bands, n_samples) = data.read_matlab(input_file)
print(f'{n_subjects=}, {n_channels=}, {n_frequency_bands=}, {n_samples=}')

It contains arrays

In [ ]:
metrics.keys()

Dimensions are

In [ ]:
metrics['GPDC_H'].shape

*Raw* frequency bands are combined into EEG bands.

In [ ]:
eeg_band_from_raw_freq = {
    'delta': range(4),
    'theta': range(4,8),
    'alpha': range(8,12),
    'beta': range(12,30),
    'gamma': range(30,50)
}

A convenience function to write a connectivity matrix in a format suitable to be plot by *pgfplots* as a heatmap.

In [ ]:
def connectivity_matrix_to_pgf(matrix: np.ndarray, output_file: Union[str, pathlib.Path], meta:str = '') -> None:

    # header
    with output_file.open('w') as f:
        f.write(f'# {meta}\n')
        f.write(f'# <source channel> <destination channel> <metric>\n')

    # data
    with output_file.open('a') as f:
        for i_coord, coord in enumerate(itertools.product(np.arange(n_channels), repeat=2), start=1):
            f.write(f'{coord[1]} {coord[0]} {matrix[coord[1],coord[0]]}\n')

            # a newline every time a column is processed
            if i_coord % n_channels == 0:
                f.write('\n')

We write files for ill and healthy subjects using the above function.

In [ ]:
# for k, meta, output in zip(['GPDC_S'], ['GPDC, alpha band, ill subject #0, sample #0'], ['connectivity_ill.txt']):
for k, meta, output in zip(
    ['GPDC_S', 'GPDC_H'],
    ['GPDC, alpha band, ill subject #0, sample #0', 'GPDC, alpha band, healthy subject #0, sample #0'],
    ['connectivity_ill.txt', 'connectivity_healthy.txt']):
    averaged_freqs = data.average_frequencies(metrics[k], eeg_band_from_raw_freq.values())
    connectivity_matrix_example = averaged_freqs[0, ..., 2, 0]
    connectivity_matrix_to_pgf(connectivity_matrix_example, output_path / output, meta)

After averaging over frequencies, these are the dimensions of the resulting arrays

In [ ]:
averaged_freqs.shape

Ultimately, the dimensions of a *single* connectiviy matrix are

In [ ]:
connectivity_matrix_example.shape

### Pre-processing

A function to build a `DataFrame` from *healthy* and *ill* samples for a given metric.

In [ ]:
def metric_to_df(healthy: np.array, ill: np.array, columns_name_prefix: str, dep_var: str = 'ill') -> pd.DataFrame:
    
    pre_processed_data = [
        data.to_df(
            *data.images_to_vectors(
                data.average_frequencies(array, eeg_band_from_raw_freq.values()),
                both=True, name_prefix=columns_name_prefix
            )
        )
        for array in [healthy, ill]
    ]
    
    pre_processed_data[0][dep_var] = False
    pre_processed_data[1][dep_var] = True
    
    return pd.concat(pre_processed_data, axis=0)

#### GPDC

The above function is applied on *GPDC* metric

In [ ]:
df_GPDC = metric_to_df(metrics['GPDC_H'], metrics['GPDC_S'], columns_name_prefix='GPDC_', dep_var=dep_var)
df_GPDC.head()

The number of features is

In [ ]:
df_GPDC.shape[1] - 2

Subjects are

In [ ]:
df_GPDC['subject'].unique()

Notice that every *subject* above actually **encompasses two different subjects** since we have `n_subjects` healthy subjects and `n_subjects` ill ones, and they are labeled the same. In other words, the $n$-th subject encompasses the samples of the $n$-th healthy subject and those of the $n$-th ill subject. Hence, the number of samples for the, e.g., $0$-th *subject* is twice the number of actual samples we have for any given subject.

In [ ]:
assert df_GPDC.groupby('subject').size().loc[0] == n_samples * 2

The above remark entails that we have the same number of *healthy* and *ill* labels.

In [ ]:
df_GPDC[dep_var].value_counts()

#### dDTF

In [ ]:
df_dDTF = metric_to_df(metrics['dDTF_H'], metrics['dDTF_S'], columns_name_prefix='dDTF_', dep_var=dep_var)
df_dDTF.head()

**CAVEAT**: notice that every *subject* in the `DataFrame` actually encompasses data from two subjects, one healthy and one sick. Moreover, this (arbitrary) pairing of subjects affects data splitting below.

#### Both metrics

In [ ]:
# `ill` columns are the same in both `DataFrame`s
assert df_GPDC['ill'].equals(df_dDTF['ill'])

# idem for `subject`
assert df_GPDC['subject'].equals(df_dDTF['subject'])

# we get rid of those in, e.g., `df_GPDC`, before concatenating
df = pd.concat((df_GPDC.drop(['ill', 'subject'], axis=1), df_dDTF), axis=1)
df.shape

In [ ]:
df.head()

The indexes in the concatenated `DataFrame`s were the same, and so we have duplicated labels

In [ ]:
df.index.nunique()

We make a new index

In [ ]:
df = df.reset_index(drop=True)

The number of features is

In [ ]:
df.shape[1] - 2

## Splitting the data into training and validation sets

A function to split the data into training and validation according to the indexes of *paired* subjects. That is, subject #3 means healthy subject #3 and ill subject #3.

In [ ]:
def split_data_on_paired_subjects(df: pd.DataFrame, i_validation_subjects: list) -> Tuple[list, list]:
    
    belong_in_validation = df['subject'].isin(i_validation_subjects)
    i_validation = np.where(belong_in_validation)[0]
    i_training = np.where(~belong_in_validation)[0]
    
    return sorted(i_training), sorted(i_validation)

An example run

In [ ]:
i_training, i_validation = split_data_on_paired_subjects(df, [0])
i_training[:4], i_validation[:4]

The number of samples in the training and validation sets. The latter must be `n_samples` $\times$ $2$ subjects (one healthy, and ill) $\times$ `len(i_validation_subjects)`.

In [ ]:
len(i_training), len(i_validation)

A function to split the data considering separately healthy and ill subjects.

In [ ]:
def split_data_on_individual_subjects(
    df: pd.DataFrame, i_healthy_subjects_in_valid: list, i_ill_subjects_in_valid: list) -> Tuple[list, list]:
    
    healthy_in_validation = (df['subject'].isin(i_healthy_subjects_in_valid)) & (~ df['ill'])
    ill_in_validation = (df['subject'].isin(i_ill_subjects_in_valid)) & df['ill']
    
    i_validation = np.nonzero((healthy_in_validation | ill_in_validation).to_numpy())[0].tolist()
    i_training = list(set(range(len(df))) - set(i_validation))
    
    assert len(df) == len(i_validation) + len(i_training)
    
    return sorted(i_training), sorted(i_validation)

An arbitrary example

In [ ]:
i_training, i_validation = split_data_on_individual_subjects(df, [0, 1], [1])

In [ ]:
i_training[:4], i_validation[:4]

The number of samples in the training and validation sets. The latter must be `n_samples` $\times$ (`len(i_healthy_subjects_in_valid)` + `len(i_ill_subjects_in_valid)`).

In [ ]:
len(i_training), len(i_validation)

## Machine Learning

Subjects in `i_testing_subjects` are excluded from the training data

In [ ]:
df_training = df.loc[~df['subject'].isin(i_testing_subjects)]
df_training

Number of ill and healthy subjects

In [ ]:
df_training[dep_var].value_counts()

Indexes for the training and validation sets

In [ ]:
i_training, i_validation = split_data_on_paired_subjects(df_training, i_validation_subjects)

The sum of samples in *training* and *validation* brings back the original size.

In [ ]:
assert (len(i_validation) + len(i_training)) == len(df_training)

Also, the indexes are disjoint

In [ ]:
assert set(i_validation).intersection(i_training) == set()

The *data columns* (as opposed to the *outcome column* or the *subject column*) can be identified because they contain numbers.

In [ ]:
data_columns = df.filter(regex=r'\d+').columns.to_list()
data_columns[:4]

A fastai *tabular object*

In [ ]:
to = TabularPandas(df_training, cont_names=data_columns, y_names=dep_var, splits=(i_training, i_validation))
type(to)

Number of samples in the *training* and *validation* sets

In [ ]:
len(to.train), len(to.valid)

First few samples

In [ ]:
to.show(3)

For the sake of convenience, we set variables for easy access to the independent and dependent variables in both the *training* and *validation* sets

In [ ]:
xs, y = to.train.xs,to.train.y
valid_xs, valid_y = to.valid.xs,to.valid.y

## Simple decision tree

In [ ]:
model = DecisionTreeClassifier(max_leaf_nodes=4)
model.fit(xs, y);

The resulting tree is drawn. Notice that we are renaming the columns so that [graphviz](https://graphviz.org/) (on which `draw_tree` relies) doesn't get confused.

In [ ]:
draw_tree(
    model, xs.rename(lambda x: x.replace('->', '').replace('_', ''), axis='columns'), leaves_parallel=True, precision=2)

In [ ]:
model

Another fit with different hyperparameters

In [ ]:
model = DecisionTreeClassifier(min_samples_leaf=5, random_state=42)
model.fit(xs, y);

How many leaves result? Is that a lot relative to the size of the dataset?

In [ ]:
model.get_n_leaves(), len(xs)

Notice that the model is **classifier** (rather than a regressor), and hence it predicts integers.

In [ ]:
model.predict(valid_xs).dtype

However, it is possible to estimate the probability of each class

In [ ]:
model.predict_proba(valid_xs)[10:15]

The corresponding *decisions* are

In [ ]:
model.predict(valid_xs)[10:15]

These are the classes (and hence the probability of a subject being *ill* is in the 2nd column)

In [ ]:
model.classes_

Accuracy on training and validation

In [ ]:
model_accuracy(model, xs, y), model_accuracy(model, valid_xs, valid_y)

## Random forests

A simple *random forest regressor* with default settings

In [ ]:
model = RandomForestRegressor()
model.fit(xs, y);

*Training* and *validation* accuracy

In [ ]:
model_accuracy(model, xs, y), model_accuracy(model, valid_xs, valid_y)

A convenience function to build and fit in one go a *random forest*. It returns the fitted model.

In [ ]:
def rf(xs, y, n_estimators=200, max_samples=None,
       max_features=0.025, min_samples_leaf=10):
    return RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators,
        max_features=max_features, max_samples=max_samples,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [ ]:
model = rf(xs, y);

In [ ]:
model_accuracy(model, xs, y), model_accuracy(model, valid_xs, valid_y)

Out-of-bag (OOB) predictions are much better

In [ ]:
# accuracy(model.oob_prediction_,y)

The prediction for every sample (in the *validation* set) from each individual tree

In [ ]:
all_trees_predictions = np.stack([m.predict(valid_xs) for m in model.estimators_])
all_trees_predictions.shape

The above validation accuracy is simply the mean across all the tress

In [ ]:
accuracy(all_trees_predictions.mean(0), valid_y)

The standard deviation across tress

In [ ]:
sds = all_trees_predictions.std(axis=0)
sds.min(), sds.max()

Prediction as we increase the number of trees

In [ ]:
n_trees = all_trees_predictions.shape[0]
n_trees_accuracy = np.array(
    [accuracy(all_trees_predictions[:i+1].mean(axis=0), valid_y) for i in range(n_trees)])

In [ ]:
plt.plot(np.arange(1, n_trees+1), n_trees_accuracy);

## Automating the above process

A convenience function to split the data into *training*, *validation* and *testing* datasets, and fit a random forest to the first one.

In [ ]:
def split_data_and_fit(df: pd.DataFrame, i_testing_subjects: list, i_validation_subjects: list):

    # the names of the columns, which are NOT the dependent variable
    data_columns = df.filter(regex=r'\d+').columns.to_list()

    # training data is filtered out of the full dataset
    df_training = df.loc[~df['subject'].isin(i_testing_subjects)]
    
    i_training, i_validation = split_data_on_paired_subjects(df_training, i_validation_subjects)

    # fastai data object
    to = TabularPandas(df_training, cont_names=data_columns, y_names=dep_var, splits=(i_training, i_validation))

    # the model is built and fitted
    model = rf(to.train.xs, to.train.y)

    return model, to

## Feature importance

A convenience function to combine the names and importance (according to the random forest) of the different features into a pandas `Series`

In [ ]:
def feature_importance_series(model, xs) -> pd.Series:
    
    return pd.Series(data=model.feature_importances_, index=xs.columns).sort_values(ascending=False)

In [ ]:
fi = feature_importance_series(model, xs)
fi.head()

Worst features are

In [ ]:
fi.sort_values()[:30]

It can be seen there is a mixture of *GPDC* and *dTFT* features.

In [ ]:
print(f'{(fi > 0).sum()} features (out of {len(fi)}) contribute something')

Let us set a threshold,

In [ ]:
# threshold = 5e-3
threshold = 1e-3

and keep only those features whose importance is above it

In [ ]:
feature_above_threshold = fi > threshold
feature_above_threshold.sum()

In [ ]:
important_features = fi[feature_above_threshold]
important_features.head()

In [ ]:
xs_important = xs[important_features.index]
xs_important.head()

In [ ]:
model = rf(xs_important, y);
model_accuracy(model, xs_important, y), model_accuracy(model, valid_xs[important_features.index], valid_y)

So, nothing really changed by dismissing quite a bunch of features, specifically,

In [ ]:
xs.shape[1] - xs_important.shape[1]

### Selecting the best features

A function that gives the accuracy when using the *best $n$ features* for every possible value of $n$

In [ ]:
def accuracy_for_n_best_features(
    feature_importance: pd.Series, xs: pd.DataFrame, y: pd.Series, valid_xs: pd.DataFrame, valid_y: pd.Series
) -> pd.Series:
    
    n_features = len(feature_importance)
    
    acc = np.empty(n_features)
    
    for n_features_minus_1 in range(n_features):
        
        to_keep = feature_importance.iloc[:n_features_minus_1+1].index

        # a model with only the relevant features
        model = rf(xs[to_keep], y)
        
        acc[n_features_minus_1] = model_accuracy(model, valid_xs[to_keep], valid_y)
    
    return pd.Series(acc, index=range(1, 1+n_features))

The *aggregated* importance of the features is used

In [ ]:
%%script false --no-raise-error
n_best_feactures_accuracy = accuracy_for_n_best_features(fi, xs, y, valid_xs, valid_y)
n_best_feactures_accuracy

In [ ]:
%%script false --no-raise-error
n_best_feactures_accuracy[:50].plot();

It seems beyond a few dozens of features, there is not much to be gained.

### Impact on performance of the feature importance threshold

A function to assess the performance when dismissing those features whose performance is below a certain threshold

In [ ]:
def importance_threshold_to_accuracy(
    feature_importance: pd.Series, threshold: float, xs: pd.DataFrame, y: pd.Series, valid_xs: pd.DataFrame,
    valid_y: pd.Series) -> float:
    
    # `True` for those features whose importance is above the treshold
    feature_above_threshold = feature_importance >= threshold
    
    # their names are elicited
    important_features = fi[feature_above_threshold].index
    
    # a new `DataFrame` for training with only the most important features
    xs_important = xs[important_features]
    
    # a model with only the relevant features
    model = rf(xs_important, y)
    
    # accuracy on the validation set
    return model_accuracy(model, valid_xs[important_features], valid_y)

In [ ]:
max_importance = fi.iloc[0].item()
max_importance

Using only the best feature

In [ ]:
importance_threshold_to_accuracy(fi, max_importance, xs, y, valid_xs, valid_y)

Over a range of thresholds

In [ ]:
# thresholds = np.linspace(max_importance / 1_000, max_importance, 10)
thresholds = np.arange(1e-3, max_importance, 5e-3)
threshold_accuracy = [importance_threshold_to_accuracy(fi, t, xs, y, valid_xs, valid_y) for t in thresholds]
threshold_n_features = (fi.values[np.newaxis, :] > thresholds[:, np.newaxis]).sum(axis=1)
threshold_accuracy_df = pd.DataFrame(
    np.c_[thresholds, threshold_accuracy, threshold_n_features],
    columns=['threshold', 'accuracy', 'n_features']
).astype({'n_features': int})
threshold_accuracy_df

We can plot the *accuracy* against the *threshold*

In [ ]:
threshold_accuracy_df.plot(x='threshold', y='accuracy');

Notice that a lower threshold (left part of the plot) entails more features being used.

### Variability on features importance

A convenience function to estimate *feature importance* for a given split of the data.

In [ ]:
def split_data_and_compute_feature_importance(i_testing_subjects: list, i_validation_subjects: list) -> pd.Series:

    model, to = split_data_and_fit(df, i_testing_subjects, i_validation_subjects)

    # a Pandas series with the importance of every feature
    return feature_importance_series(model, to.xs)

With the same data splitting considered so far

In [ ]:
fi_1 = split_data_and_compute_feature_importance(i_testing_subjects, i_validation_subjects)
fi_1

With a different data split

In [ ]:
fi_2 = split_data_and_compute_feature_importance([0, 1], [2, 3])
fi_2

Both results are assembled in a single `DataFrame` (properly pairing together values referred to the same feature)

In [ ]:
fi_df = pd.concat((fi_1, fi_2), axis=1)
fi_df.head()

The most *overall* important features when considering the two splits

In [ ]:
fi_df.sum(axis=1).sort_values(ascending=False)

We compute the *features importance* for every possible data split of the *training* set into *training* and *validation*, while excluding the *testing* subjects.

In [ ]:
fis = [
    split_data_and_compute_feature_importance(i_testing_subjects, list(i_valid))
    for i_valid in itertools.combinations(range(12), 2)
]

In [ ]:
fis_df = pd.concat(fis, axis=1)
fis_df.head()

Again, the most *overall* important features

In [ ]:
fis_df.sum(axis=1).sort_values(ascending=False)[:20]

The top features are not the same as before, but those that were important before (*GPDC* $116$ and $156$, *dDTF* $79$...) still are.

Besides the mean, we can also compute the standard deviation

In [ ]:
fis_df.std(axis=1).sort_values(ascending=False)

Standard deviations are pretty small relative to the means.

### Another take

A function to split the data on individual subjects (as opposed to *paired* subjects) and fit the model.

In [ ]:
def split_data_on_individual_subjects_and_fit(
    df: pd.DataFrame, i_healthy_subjects_in_valid: list, i_ill_subjects_in_valid: list):

    # the names of the columns, which are NOT the dependent variable
    data_columns = df.filter(regex=r'\d+').columns.to_list()
    
    i_training, i_validation = split_data_on_individual_subjects(
        df, i_healthy_subjects_in_valid, i_ill_subjects_in_valid)

    # fastai data object
    to = TabularPandas(df, cont_names=data_columns, y_names=dep_var, splits=(i_training, i_validation))

    # the model is built and fitted
    model = rf(to.train.xs, to.train.y)

    return model, to

A new function that encapsulates the application of the above function on every combination of *individual* subjects for `i_healthy_subjects_in_valid` and `i_ill_subjects_in_valid`. The names of the columns in the resulting `DataFrame` indicate the index of the healthy subject and that of the ill one.

In [ ]:
def fi_individual_combinations():
    
    series = []
    names = []

    for i_healthy, i_ill in itertools.product(range(n_subjects), repeat=2):

        # split and fitting
        m, t = split_data_on_individual_subjects_and_fit(df, [i_healthy], [i_ill])

        # results are recorded
        series.append(feature_importance_series(m, t.xs))
        names.append(f'{i_healthy}H{i_ill}I')

    df_fi = pd.concat(series, axis=1)
    df_fi.columns = names
    
    return df_fi

In [ ]:
%%script false --no-raise-error
df_fi = fi_individual_combinations()
df_fi

On average

In [ ]:
%%script false --no-raise-error
average_fi = df_fi.mean(axis=1).sort_values(ascending=False)
average_fi

## Performance

A *sub*directory for the results

In [ ]:
subject_aware_output_path = output_path / 'subject_aware'
subject_aware_output_path.mkdir(exist_ok=True)

For the sake of convenience, a `DataFrame` focused only on subjects and whether or not they are ill

In [ ]:
subjects_df = df.groupby(['subject', 'ill']).first().reset_index()[['subject', 'ill']]
subjects_df

K-fold cross-validation

In [ ]:
k_folder = StratifiedKFold(n_splits=n_folds_cross_validation)

In [ ]:
res_metrics = []
res_index = []
res_sensitivity_specificity_auc = []
res_feature_importance = []

for _, i_valid in k_folder.split(subjects_df['subject'], subjects_df['ill']):
    
    # the subjects that will go into the validation set
    valid_subjects = subjects_df.loc[i_valid.tolist()]['subject'].values
    
    # every subject is present twice (healthy and ill) in the validation set
    assert (np.array(list(collections.Counter(valid_subjects).values())) == 2).all()
    
    # the valid subjects without duplicates
    unique_valid_subjects = np.unique(valid_subjects)
    
    # the data is split and model fit on the training set
    model, to = split_data_and_fit(df, [], unique_valid_subjects)
    
    # training and validation sets are extracted from tabular object `to`
    xs, y = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    
    # accuracy on the training and validation sets
    res_metrics.append((model_accuracy(model, xs, y), model_accuracy(model, valid_xs, valid_y)))
    
    # a label for the index
    res_index.append(' & '.join([str(e) for e in unique_valid_subjects]))
    
    # sensitivity, specificity and AUC
    res_sensitivity_specificity_auc.append(sensitivity_specificity_auc(model.predict_proba(valid_xs)[:, 1], valid_y))
    
    # feature importance
    res_feature_importance.append(feature_importance_series(model, xs))

AUCs = [e[2] for e in res_sensitivity_specificity_auc]
metrics = pd.DataFrame(np.c_[res_metrics, AUCs], columns=['training', 'validation', 'AUC'])
metrics.index = res_index
metrics.index.name = 'sujects in validation'
metrics

The average on the validation set

In [ ]:
metrics['validation'].mean()

Average AUC is

In [ ]:
metrics['AUC'].mean()

A convenience function to write the (ordered) features importance into a *csv* file.

In [ ]:
def write_importance_to_csv(
    features_importance: pd.Series, output_file: Union[str, pathlib.Path], n_rows: Optional[int] = 10) -> None:

    with output_file.open('w') as f:
        
        f.write('# feature, importance\n')

    features_importance[:n_rows][::-1].to_csv(output_file, mode='a', header=False)

A class to output features' importance results

In [ ]:
class FeaturesImportanceManager:
    
    def __init__(self, colors: set) -> None:
        
        self.colors = colors.copy()
        self.feature_name_to_color = {}
    
    def summarize_and_write_importance_to_csv(
        self, features_importance: list, output_path: Union[str, pathlib.Path], n_best: int = 10
    ) -> Tuple[pd.DataFrame, set]:
        """
        Processes a list of features importance `Series` (one per fold), and write the corresponding *csv* files.
        """

        # in case a `str` was passed
        output_path = pathlib.Path(output_path)

        # in case the directory doesn't exist
        output_path.mkdir(exist_ok=True)

        # a `DataFrame` with all the importance `Series`
        features_importance_df = pd.concat(res_feature_importance, axis=1)

        # a function to use along `replace` below
        def repl(m):
            return f'{m.group(2)}/' + \
                list(eeg_band_from_raw_freq)[int(m.group(1))] + \
                f' ch. {eeg_index_to_name[int(m.group(3))-1]} to {eeg_index_to_name[int(m.group(4))-1]}'

        # labels in the index are renamed for the sake of *human* readability
        features_importance_df.index = features_importance_df.index.str.replace(
            r'f(\d)_([a-zA-Z]+)_(\d+)->(\d+)', repl, regex=True)

        # for every feature we find
        features_importance_summary_df = pd.concat([
            # ...the minimum importance value across all the folds, and
            features_importance_df.min(axis=1).rename('Minimum'),
            # ... also the average
            features_importance_df.mean(axis=1).rename('Average')
        ], axis=1)
        
        #
        self.features_importance_summary_df = features_importance_summary_df

        # the `n_best` features in every case
        best_min_features_importance = features_importance_summary_df['Minimum'].sort_values(ascending=False)[:n_best]
        best_average_features_importance = features_importance_summary_df['Average'].sort_values(
            ascending=False)[:n_best]

        # features that are in both rankings
        common_features = set(best_min_features_importance.index).intersection(
            set(best_average_features_importance.index))
        
        new_colors = common_features - set(self.feature_name_to_color.keys())

        # there are enough different colors
        assert len(new_colors) <= len(self.colors)

        # a mapping from feature to color
        mapping_update = dict(zip(sorted(list(new_colors)), sorted(list(self.colors))))
        
        self.colors -= set(mapping_update.values())
        
        self.feature_name_to_color.update(mapping_update)

        # a function that adds color to any label in the `Series`' index that is in the above mapping
        def color_features_names(fi: pd.Series) -> pd.Series:

            return fi.set_axis(
                [f'\\textcolor{{{self.feature_name_to_color[n]}}}{{\\textbf{{{n}}}}}'
                 if n in self.feature_name_to_color
                 else f'\\textcolor{{gray}}{{{n}}}' for n in fi.index])

        write_importance_to_csv(
            color_features_names(best_min_features_importance), output_path / 'minimum_importance.txt', n_rows=None)
        write_importance_to_csv(
            color_features_names(best_average_features_importance),
            output_path / 'average_importance.txt', n_rows=None)

        return features_importance_df, common_features

features_importance_manager = FeaturesImportanceManager(latex_colors)

*csv*'s with features importance are written

In [ ]:
subject_aware_feature_importance_df, subject_aware_common_features = features_importance_manager.summarize_and_write_importance_to_csv(res_feature_importance, subject_aware_output_path)

The *average* across folds for every feature

In [ ]:
subject_aware_feature_importance_mean = subject_aware_feature_importance_df.mean(axis=1)
subject_aware_feature_importance_mean

A *bar plot*

In [ ]:
# subject_aware_feature_importance_mean.plot.bar();

In [ ]:
subject_aware_common_features

These features account for this percentage of *importance*

In [ ]:
subject_aware_feature_importance_mean.loc[subject_aware_common_features].sum() * 100

while making up this percentage out of the overall number of features

In [ ]:
len(subject_aware_common_features) / subject_aware_feature_importance_df.shape[0] * 100

In [ ]:
features_importance_manager.features_importance_summary_df

In [ ]:
features_importance_manager.colors

A function to save the ROC curves into *csv* files

In [ ]:
def write_rocs_to_csvs(sensitivity_specificity_auc: list, output_path: Union[str, pathlib.Path]) -> None:
    
    for i_fold, (specificity, sensitivity, _) in enumerate(sensitivity_specificity_auc):

        output_file = output_path / f'fold_{i_fold}.txt'

        # header
        with output_file.open('w') as f:
            f.write('# 1-specificity, sensitivity\n')
        
        # data
        with output_file.open('a') as f:
            np.savetxt(f, np.stack([1. - specificity, sensitivity]).T)

It is used on the above results

In [ ]:
write_rocs_to_csvs(res_sensitivity_specificity_auc, subject_aware_output_path)

Another convenience function, this one to write the AUCs to a (single) *csv*

In [ ]:
def write_auc_to_csv(auc: np.array, output_path: Union[str, pathlib.Path]) -> None:

    output_file = output_path / 'AUC.txt'

    # header
    with output_file.open('w') as f:
        f.write('# AUC\n')
    
    # data
    with output_file.open('a') as f:
        np.savetxt(f, auc)

In [ ]:
write_auc_to_csv(metrics['AUC'].values, subject_aware_output_path)

### Without splitting samples by subject

A *sub*directory for the results

In [ ]:
subject_unaware_output_path = output_path / 'subject_unaware'
subject_unaware_output_path.mkdir(exist_ok=True)

An object that will produce an iterator to go through the different splits

In [ ]:
folder = StratifiedKFold(n_splits=n_folds_cross_validation)

The first subject (for instance) is used as a model for exacting training and testing *indexes* *all* the subjects. This is accomplished using the `nth` method of a *grouped* object.

In [ ]:
first_subject_df = df[df['subject']==0]

Data is grouped by *subject*

In [ ]:
grouped_by_subject = df.groupby('subject', as_index=False)

We fit and validate for every fold

In [ ]:
res_metrics = []
res_indexes = []
res_sensitivity_specificity_auc = []
res_feature_importance = []


for i_train, i_valid in folder.split(first_subject_df.index, first_subject_df['ill']):
    
    splits = (grouped_by_subject.nth(i_train.tolist()).index.tolist(),
              grouped_by_subject.nth(i_valid.tolist()).index.tolist())
    
    to = TabularPandas(df, cont_names=data_columns, y_names=dep_var, splits=splits)
    
    xs, y = to.train.xs, to.train.y
    valid_xs, valid_y = to.valid.xs, to.valid.y
    
    model = rf(xs, y)
    
    # feature importance
    res_feature_importance.append(feature_importance_series(model, xs))
    
    res_metrics.append([model_accuracy(model, xs, y), model_accuracy(model, valid_xs, valid_y)])
    res_indexes.append(splits)
    
    res_sensitivity_specificity_auc.append(sensitivity_specificity_auc(model.predict_proba(valid_xs)[:, 1], valid_y))

auc = [r[2] for r in res_sensitivity_specificity_auc]
folded_accuracy_df = pd.DataFrame(np.c_[res_metrics, auc], columns=['training', 'validation', 'AUC'])
del res_metrics

The results:

In [ ]:
folded_accuracy_df

The mean in the validation set is

In [ ]:
folded_accuracy_df['validation'].mean()

*csv*'s with features importance are written

In [ ]:
subject_unaware_feature_importance, subject_unaware_common_features = features_importance_manager.summarize_and_write_importance_to_csv(
    res_feature_importance, subject_unaware_output_path)

The AUC values are written to a file

In [ ]:
subject_unaware_output_path

In [ ]:
write_auc_to_csv(folded_accuracy_df['AUC'].values, subject_unaware_output_path)

*Receiver Operating Characteristic* (ROC) for every fold

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for sensitivity, specificy, _ in res_sensitivity_specificity_auc:
    ax.plot(1.-specificy, sensitivity)
ax.set_xlim((-0.01, 0.5));

The results for every fold are saved in a separate file

In [ ]:
write_rocs_to_csvs(res_sensitivity_specificity_auc, subject_unaware_output_path)

Quick check: every index made into the validation set exactly once.

In [ ]:
# the indexes in the validation sets in all the folds
i_validation_aggregated = sum([e[1] for e in res_indexes], [])

# every element in the above list is different, and the number of elements is exactly the overall number of samples
assert len(i_validation_aggregated) == len(np.unique(i_validation_aggregated)) == len(df)

The sizes of the training and validation sets in every fold

In [ ]:
folds_df = pd.DataFrame([[len(i_t), len(i_v)] for i_t, i_v in res_indexes], columns=['training', 'validation'])
folds_df

The number of samples per subject and that of folds for cross-validation may yield an odd number of samples (from every subject) in the validation set.

In [ ]:
n_samples_per_subject = grouped_by_subject.size()['size'].iloc[0]
n_samples_per_subject / n_folds_cross_validation

If that's the case, we will not have the same number of healthy and ill examples (for every subject) nor in the training set

In [ ]:
grouped_by_subject.nth(i_train.tolist()).groupby(['subject', 'ill']).size()

nor in the validation set

In [ ]:
grouped_by_subject.nth(i_valid.tolist()).groupby(['subject', 'ill']).size()

(numbers above refer to the last fold since we are using the last state of the above loop).

## Aware vs. Unaware

In [ ]:
always_relevant_features = subject_aware_common_features.intersection(subject_unaware_common_features)
always_relevant_features

*Left-over* colors

In [ ]:
features_importance_manager.colors